# Un problema común de trabajar con Valores Faltantes

Suponga que tiene un conjunto de datos a los cuales aplica modelos de Regresión Lineal para modelar la relación entre variables, generar predicciones y entender cómo una variable afecta la otra. Crea varios modelos lineales con diferentes variables y los comparas para determinar el que mejor se ajusta a los datos. Eliges uno de ellos y al utilizarlo para predecir el valor de una variable en función de otra, arroja valores que no corresponden con la realidad o que están fuera del rango de la variable. 

Qué sucedió? No te percataste de la presencia de valores faltantes!. Es un problema muy común que se presenta al utilizar modelos y que puedes presenciar solo al final de todo el proceso debido a que Python corre los algoritmos sin errores y sin advertir sobre ellos.

La solución es realizar un tratamiento de los valores faltantes al iniciar el análisis exploratorio de los datos. En lo que sigue te muestro cómo se genera este problema para que lo tengas en cuenta en tus análisis.

## Librerías

In [5]:
import janitor
import numpy as np  
import pandas as pd 
import scipy.stats  # trabjar con estadisticos
import statsmodels.api as sm
import statsmodels.formula.api as smf 

## Cargar los Datos

Cargamos un dataset de prueba sobre la calidad del aire proporcionado por la libreria statsmodels

In [6]:
airquality_df = (
    sm.datasets.get_rdataset('airquality')      # obtener datos
    .data                                       # acceder a los datos
    .clean_names(                               # cambiar formato de nombre de las variables
        case_type='snake'
    )
    .add_column('year',1973)                    # agregar nueva columna
    .assign(                                    # nueva columna con fecha
        date = lambda df: pd.to_datetime(df[['year', 'month', 'day']])
    )
    .sort_values(by='date')                     # ordenar por fecha
    .set_index('date')                          # indice fecha
)

airquality_df

,ozone,solar_r,wind,temp,month,day,year
date,,,,,,,
1973-05-01,41.0,190.0,7.4,67,5,1,1973
1973-05-02,36.0,118.0,8.0,72,5,2,1973
1973-05-03,12.0,149.0,12.6,74,5,3,1973
1973-05-04,18.0,313.0,11.5,62,5,4,1973
1973-05-05,NaN,NaN,14.3,56,5,5,1973
...,...,...,...,...,...,...,...
1973-09-26,30.0,193.0,6.9,70,9,26,1973
1973-09-27,NaN,145.0,13.2,77,9,27,1973
1973-09-28,14.0,191.0,14.3,75,9,28,1973


## Variables con Valores Faltantes

Verificamos las variables que contienen valores faltantes y hacemos un conteo de los mismos

In [7]:
(
    airquality_df
    .isna()
    .sum()
)

ozone      37
solar_r     7
wind        0
temp        0
month       0
day         0
year        0
dtype: int64

Vemos que solo dos variables poseen valores faltantes. Ozone posee 37 y solar_r posee 7.

## Modelo de Regresión Lineal sobre las variables con valores faltantes

Vamos a ajustar un modelo de regresión lineal (OLS) con statmodels sobre las variables temp y ozone que contiene valores faltantes.

In [8]:
# modelo de regresion lineal de temp sobre ozone
(
    smf.ols(
        formula='temp ~ ozone',     # variables seleccionadas
        data= airquality_df         # DataFrame
    )
    .fit()                          # ajuste del modelo
    .summary()                      # resumen de valores del modelo
    .tables[0]                      # mostrar valores en formato tabular
)

Dep. Variable:,temp,R-squared:,0.488
Model:,OLS,Adj. R-squared:,0.483
Method:,Least Squares,F-statistic:,108.5
Date:,"Sun, 30 Jul 2023",Prob (F-statistic):,2.93e-18
Time:,08:29:41,Log-Likelihood:,-386.27
No. Observations:,116,AIC:,776.5
Df Residuals:,114,BIC:,782.1
Df Model:,1,,
Covariance Type:,nonrobust,,


El modelo OLS se aplica sobre 116 observaciones de las 157 existentes lo que indica que no tomo en cuenta los datos de ozone con valores faltantes. El R-squared es un indicador del ajuste del modelo sobre los datos. Se obtuvo un valor de 0.488

Ahora al modelo anterior agregamos la variable solar_r que contiene valores faltantes. Es importante entender que a medida que agregamos mas variables a un modelo este tiende a ajustarse mejor a los datos debido a que hay mas informacion que lo alimenta. Veamos datos arrojados por el nuevo ajuste

In [9]:
# modelo de regresion lineal de temp sobre ozone y solar_r
(
    smf.ols(
        formula='temp ~ ozone + solar_r',     # variables seleccionadas
        data= airquality_df                   # DataFrame
    )
    .fit()                                    # ajuste del modelo
    .summary()                                # resumen de valores del modelo
    .tables[0]                                # mostrar valores en formato tabular
)

Dep. Variable:,temp,R-squared:,0.491
Model:,OLS,Adj. R-squared:,0.481
Method:,Least Squares,F-statistic:,52.07
Date:,"Sun, 30 Jul 2023",Prob (F-statistic):,1.47e-16
Time:,08:29:41,Log-Likelihood:,-369.78
No. Observations:,111,AIC:,745.6
Df Residuals:,108,BIC:,753.7
Df Model:,2,,
Covariance Type:,nonrobust,,


Este modelo se aplica sobre 111 observaciones, mucho menos que en el anterior debido a que no toma en cuenta los datos con valores faltantes de ozone y solar_r. El modelo arroja un valor de R-squared de 0.491.

El siguiente paso seria comparar las R-squared de ambos modelos y determinar cual de los dos se ajusta mejor. Segun los datos, parece que el segundo modelo se ajuta mejor debido a que su R es mayor. Sin embargo, compararlos es un error debido a que los modelos se aplican sobre diferentes observaciones.

Esto es un error muy comun que se comete gracias a que estos modelos corren sobre la data a pesar de contener valores faltantes. Tenerlo en cuenta te ayudara a evitar trabajar en vano y a considerar los valores faltantes al inicio del analisis. 